# iprPy calculation basics

In [23]:
from IPython.display import display, Markdown

import numpy as np

import atomman as am
import atomman.unitconvert as uc

import iprPy

In [8]:
lammps_command = 'lmp_serial'
mpi_command = None
#mpi_command = 'mpiexec -localonly 6'

## 1. Load a calculation object

Calculation objects provide the tools to run and analyze iprPy calculations.

You can see the names of all current working calculations using the calculationmanager.

In [50]:
iprPy.calculationmanager.loaded_style_names

['bond_angle_scan',
 'crystal_space_group',
 'diatom_scan',
 'diffusion_liquid',
 'diffusion_msd',
 'dislocation_dipole',
 'dislocation_monopole',
 'dislocation_periodic_array',
 'dislocation_SDVPN',
 'elastic_constants_dynamic',
 'elastic_constants_static',
 'energy_check',
 'E_vs_r_scan',
 'free_energy',
 'free_energy_liquid',
 'grain_boundary_grip',
 'grain_boundary_static',
 'isolated_atom',
 'melting_temperature',
 'phonon',
 'point_defect_diffusion',
 'point_defect_mobility',
 'point_defect_static',
 'relax_box',
 'relax_dynamic',
 'relax_liquid',
 'relax_liquid_redo',
 'relax_static',
 'stacking_fault_map_2D',
 'stacking_fault_static',
 'surface_energy_static',
 'viscosity_driving',
 'viscosity_green_kubo']

In [21]:
calc = iprPy.load_calculation('elastic_constants_static')

## 2. Run calculation functions directly

You can access and run a calculation's primary function directly as the **calc()** method.

In [4]:
print(calc.calc.__doc__)


    Repeatedly runs the ELASTIC example distributed with LAMMPS until box
    dimensions converge within a tolerance.
    
    Parameters
    ----------
    lammps_command :str
        Command for running LAMMPS.
    system : atomman.System
        The system to perform the calculation on.
    potential : atomman.lammps.Potential
        The LAMMPS implemented potential to use.
    mpi_command : str, optional
        The MPI command for running LAMMPS in parallel.  If not given, LAMMPS
        will run serially.
    strainrange : float, optional
        The small strain value to apply when calculating the elastic
        constants (default is 1e-6).
    etol : float, optional
        The energy tolerance for the structure minimization. This value is
        unitless. (Default is 0.0).
    ftol : float, optional
        The force tolerance for the structure minimization. This value is in
        units of force. (Default is 0.0).
    maxiter : int, optional
        The maximum number of

In [22]:
potential = am.load_lammps_potential('2004--Zhou-X-W--Au--LAMMPS--ipr1', getfiles=True)

ucell = am.load('crystal', potential=potential, family='A1--Cu--fcc', symbols='Au')

results = calc.calc(lammps_command, ucell, potential)

Cij = results['C'].Cij

with np.printoptions(precision=3):
    print(uc.get_in_units(Cij, 'GPa'))

[[187.919 157.751 157.751   0.      0.      0.   ]
 [157.751 187.919 157.751   0.      0.      0.   ]
 [157.751 157.751 187.919   0.      0.      0.   ]
 [  0.      0.      0.     42.542   0.      0.   ]
 [  0.      0.      0.      0.     42.542   0.   ]
 [  0.      0.      0.      0.      0.     42.542]]


## 3. Run calculations using an input file

The calculations all also support reading in the necessary inputs from a text key-value input file.

In [27]:
# Create JSON files for the potential and ucell
with open(f'{potential.name}.json', 'w') as f:
    potential.model.json(fp=f, indent=4)

ucell.dump('system_model', f='ucell.json')


View the recognized input terms with template and templatedoc

In [24]:
display(Markdown(calc.templatedoc))

# elastic_constants_static Input Terms

## Calculation Metadata

Specifies metadata descriptors common to all calculation styles.

- __branch__: A metadata group name that the calculation can be parsed by. Primarily meant for differentiating runs with different settings parameters.

## LAMMPS and MPI Commands

Specifies the external commands for running LAMMPS and MPI.

- __lammps_command__: The path to the executable for running LAMMPS on your system. Don't include command line options.
- __mpi_command__: The path to the MPI executable and any command line options to use for calling LAMMPS to run in parallel on your system. LAMMPS will run as a serial process if not given.

## Interatomic Potential

Specifies the interatomic potential to use and the directory where any associated parameter files are located.

- __potential_file__: The path to the potential_LAMMPS or potential_LAMMPS_KIM record that defines the interatomic potential to use for LAMMPS calculations.
- __potential_kim_id__: If potential_file is a potential_LAMMPS_KIM record, this allows for the specification of which version of the KIM model to use by specifying a full kim model id.  If not given, the newest known version of the kim model will be assumed.
- __potential_kim_potid__: Some potential_LAMMPS_KIM records are associated with multiple potential entries.  This allows for the clear specification of which potential (by potid) to associate with those kim models.This will affect the list of available symbols for the calculation.
- __potential_dir__: The path to the directory containing any potential parameter files (eg. eam.alloy setfl files) that are needed for the potential. If not given, then any required files are expected to be in the working directory where the calculation is executed.

## Initial System Configuration

Specifies the file and options to load for the initial atomic configuration.

- __load_file__: The path to the initial configuration file to load.
- __load_style__: The atomman.load() style indicating the format of the load_file.
- __load_options__: A space-delimited list of key-value pairs for optional style-specific arguments used by atomman.load().
- __family__: A metadata descriptor for relating the load_file back to the original crystal structure or prototype that the load_file was based on.  If not given, will use the family field in load_file if load_style is 'system_model', or the file's name otherwise.
- __symbols__: A space-delimited list of the potential's atom-model symbols to associate with the loaded system's atom types.  Required if load_file does not contain symbol/species information.
- __box_parameters__: Specifies new box parameters to scale the loaded configuration by. Can be given either as a list of three or six numbers: 'a b c' for orthogonal boxes, or 'a b c alpha beta gamma' for triclinic boxes. The a, b, c parameters are in units of length and the alpha, beta, gamma angles are in degrees.

## System Manipulations

Performs simple manipulations on the loaded initial system.

- __a_uvw__: The Miller(-Bravais) crystal vector relative to the loaded system to orient with the a box vector of a resulting rotated system. Specified as three or four space-delimited numbers. Either all or none of the uvw parameters must be given.
- __b_uvw__: The Miller(-Bravais) crystal vector relative to the loaded system to orient with the b box vector of a resulting rotated system. Specified as three or four space-delimited numbers. Either all or none of the uvw parameters must be given.
- __c_uvw__: The Miller(-Bravais) crystal vector relative to the loaded system to orient with the c box vector of a resulting rotated system. Specified as three or four space-delimited numbers. Either all or none of the uvw parameters must be given.
- __atomshift__: A rigid-body shift vector to apply to all atoms in the rotated configuration.  Specified as three space-delimited numbers that are relative to the size of the system after rotating, but before sizemults have been applied. This allows for the same relative shift of similar systems regardless of box_parameters and sizemults. Default value is '0.0 0.0 0.0' (i.e. no shift).
- __sizemults__: Multiplication parameters to construct a supercell from the rotated system.  Given as either three or six space-delimited integers. For three integers, each value indicates the number of replicas to make along the corresponding a, b, c box vector with negative values replicating in the negative Cartesian space. For six integers, the values are divided into three pairs with each pair indicating the number of 'negative' and 'positive' replications to make for a given a, b, c box vector.

## LAMMPS Energy/Force Minimization

Specifies the parameters and options associated with performing an energy and/or force minimization in LAMMPS.

- __energytolerance__: The energy tolerance to use for the minimization. This value is unitless and corresponds to the etol term for the LAMMPS minimize command. Default value is 0.0.
- __forcetolerance__: The force tolerance to use for the minimization. This value is in force units and corresponds to the ftol term for the LAMMPS minimize command. Default value is '0.0 eV/angstrom'.
- __maxiterations__: The maximum number of iterations to use for the minimization. This value corresponds to the maxiter term for the LAMMPS minimize command. Default value is 100000.
- __maxevaluations__: The maximum number of iterations to use for the minimization. This value corresponds to the maxeval term for the LAMMPS minimize command. Default value is 1000000.
- __maxatommotion__: The maximum distance that any atom can move during a minimization iteration. This value is in units length and corresponds to the dmax term for the LAMMPS min_modify command. Default value is '0.01 angstrom'.

## Input/Output Units

Specifies the default units to use for the other input keys and to use for saving to the results file.

- __length_unit__: The unit of length to use. Default value is 'angstrom'.
- __pressure_unit__: The unit of pressure to use.  Default value is 'GPa'.
- __energy_unit__: The unit of energy to use.  Default value is 'eV'.
- __force_unit__: The unit of force to use.  Default value is 'eV/angstrom'.

## Run Parameters

- __strainrange__: The strain range to apply to the system to evaluate the elastic constants.  Default value is '1e-6'

In [25]:
print(calc.template)

# Input script for iprPy calculation elastic_constants_static

# Calculation Metadata
branch                          <branch>

# LAMMPS and MPI Commands
lammps_command                  <lammps_command>
mpi_command                     <mpi_command>

# Interatomic Potential
potential_file                  <potential_file>
potential_kim_id                <potential_kim_id>
potential_kim_potid             <potential_kim_potid>
potential_dir                   <potential_dir>

# Initial System Configuration
load_file                       <load_file>
load_style                      <load_style>
load_options                    <load_options>
family                          <family>
symbols                         <symbols>
box_parameters                  <box_parameters>

# System Manipulations
a_uvw                           <a_uvw>
b_uvw                           <b_uvw>
c_uvw                           <c_uvw>
atomshift                       <atomshift>
sizemults                       <siz

Copy template and fill in values.  

Basic parsing rules
- Anything in a line after the first # is treated as a comment.
- Only lines with 2 or more terms are read in - keys without values are ignored.
- Keys can only have 1 value assigned to them.
- Any keys without values will use the default values if available.

In [29]:
infile = """
# Input script for iprPy calculation elastic_constants_static

# Calculation Metadata
branch                          

# LAMMPS and MPI Commands
lammps_command                  lmp_serial
mpi_command                     

# Interatomic Potential
potential_file                  2004--Zhou-X-W--Au--LAMMPS--ipr1.json
potential_kim_id                
potential_kim_potid             
potential_dir                   2004--Zhou-X-W--Au--LAMMPS--ipr1

# Initial System Configuration
load_file                       ucell.json
load_style                      system_model
load_options                    
family                          
symbols                         
box_parameters                  

# System Manipulations
a_uvw                           
b_uvw                           
c_uvw                           
atomshift                       
sizemults                       

# LAMMPS Energy/Force Minimization
energytolerance                 
forcetolerance                 
maxiterations                   
maxevaluations                  
maxatommotion                   

# Input/Output Units
length_unit                     
pressure_unit                   
energy_unit                     
force_unit                      

# Run Parameters
strainrange                     
"""

Calculations can be performed this way either from within Python by passing the script (or dict equivalent) to a calculation object's **run()** function.  When finished, it will generate a "results.json" file if results_json=True.


Alternatively, if you save the script as "calc_{calc_name}.in", you can run it from the command line using "iprPy run calc_{calc_name}.in".

In [33]:
calc.run(infile, results_json=True)

In [37]:
calc.C

In [36]:
with np.printoptions(precision=3):
    print(uc.get_in_units(calc.C.Cij, 'GPa'))

[[187.919 157.751 157.751   0.      0.      0.   ]
 [157.751 187.919 157.751   0.      0.      0.   ]
 [157.751 157.751 187.919   0.      0.      0.   ]
 [  0.      0.      0.     42.542   0.      0.   ]
 [  0.      0.      0.      0.     42.542   0.   ]
 [  0.      0.      0.      0.      0.     42.542]]


In [40]:
with open('results.json') as f:
    print(f.read())

{
    "calculation-elastic-constants-static": {
        "key": "c2737259-e7f1-4f3e-b861-9b53fc0e1107",
        "calculation": {
            "iprPy-version": "0.11.7",
            "atomman-version": "1.5.1",
            "LAMMPS-version": "3 Mar 2020",
            "branch": "main",
            "run-parameter": {
                "size-multipliers": {
                    "a": [
                        0,
                        3
                    ],
                    "b": [
                        0,
                        3
                    ],
                    "c": [
                        0,
                        3
                    ]
                },
                "energytolerance": 0.0,
                "forcetolerance": {
                    "value": 1e-06,
                    "unit": "eV/angstrom"
                },
                "maxiterations": 100000,
                "maxevaluations": 1000000,
                "maxatommotion": {
                    "value": 0.

In [41]:
newcalc = iprPy.load_calculation('elastic_constants_static', model='results.json')

In [42]:
with np.printoptions(precision=3):
    print(uc.get_in_units(newcalc.C.Cij, 'GPa'))

[[187.919 157.751 157.751   0.      0.      0.   ]
 [157.751 187.919 157.751   0.      0.      0.   ]
 [157.751 157.751 187.919   0.      0.      0.   ]
 [  0.      0.      0.     42.542   0.      0.   ]
 [  0.      0.      0.      0.     42.542   0.   ]
 [  0.      0.      0.      0.      0.     42.542]]


## 4. Explore the public database

In [46]:
display(Markdown(calc.querydoc))

# calculation_elastic_constants_static Query Parameters

- __key__ (*str or list, optional*): search by calculation's UUID key
- __iprPy_version__ (*str or list, optional*): search by iprPy version used
- __atomman_version__ (*str or list, optional*): search by atomman version used
- __branch__ (*str or list, optional*): search by calculation branch name
- __status__ (*str or list, optional*): search by calculation status
- __lammps_version__ (*str or list, optional*): search by LAMMPS version
- __potential_LAMMPS_key__ (*str or list, optional*): search by potential implementation's UUID key
- __potential_LAMMPS_id__ (*str or list, optional*): search by potential implementation's id
- __potential_key__ (*str or list, optional*): search by potential's UUID key
- __potential_id__ (*str or list, optional*): search by potential's id
- __load_file__ (*str or list, optional*): search by the filename for the initial configuration
- __parent_key__ (*str or list, optional*): search by the key of the parent record
- __family__ (*str or list, optional*): search by the configuration family: original prototype or crystal
- __symbol__ (*str or list, optional*): search by atomic symbols in the configuration
- __composition__ (*str or list, optional*): search by the composition of the initial configuration
- __a_mult1__ (*int or list, optional*): search by lower a_mult value
- __a_mult2__ (*int or list, optional*): search by upper a_mult value
- __b_mult1__ (*int or list, optional*): search by lower b_mult value
- __b_mult2__ (*int or list, optional*): search by upper b_mult value
- __c_mult1__ (*int or list, optional*): search by lower c_mult value
- __c_mult2__ (*int or list, optional*): search by upper c_mult value
- __strainrange__ (*float or list, optional*): search by strain range used


In [43]:
remote = iprPy.load_database('potentials')

In [44]:
records, records_df = remote.get_records('calculation_elastic_constants_static',
                                         potential_LAMMPS_id = '2004--Zhou-X-W--Au--LAMMPS--ipr1',
                                         family = 'A1--Cu--fcc',
                                         return_df = True)

In [47]:
records_df

,name,key,url,iprPy_version,atomman_version,script,branch,status,lammps_version,potential_LAMMPS_key,...,c_uvw,energytolerance,forcetolerance,maxiterations,maxevaluations,maxatommotion,strainrange,C,raw_Cij_negative,raw_Cij_positive
0,0a7f0d31-ca80-49db-87fe-a2e63363b39b,0a7f0d31-ca80-49db-87fe-a2e63363b39b,https://potentials.nist.gov/pid/rest/local/pot...,0.10.b,1.3.2,calc_elastic_constants_static,main,finished,3 Mar 2020,dff08f9e-e9c4-4b33-b64a-4a862d5dcf8c,...,"[0, 0, 1]",0.0,0.000001,5000,10000,0.01,1.000000e-07,[[1.17288 0.98474 0.98474 0. 0. 0. ...,"[[1.1728775462844674, 0.9847350794032063, 0.98...","[[1.1728767763317896, 0.9847352697067085, 0.98..."
1,211a6bfe-f0e2-4ab7-a1b0-2358df66bfab,211a6bfe-f0e2-4ab7-a1b0-2358df66bfab,https://potentials.nist.gov/pid/rest/local/pot...,0.10.b,1.3.2,calc_elastic_constants_static,main,finished,3 Mar 2020,dff08f9e-e9c4-4b33-b64a-4a862d5dcf8c,...,"[0, 0, 1]",0.0,0.000001,5000,10000,0.01,1.000000e-08,[[ 1.17288e+00 9.84735e-01 9.84735e-01 0.00...,"[[1.1728772134880685, 0.9847351905634736, 0.98...","[[1.172877129226586, 0.9847351849473062, 0.984..."
2,6607d369-4003-4603-9d22-6124c89fb456,6607d369-4003-4603-9d22-6124c89fb456,https://potentials.nist.gov/pid/rest/local/pot...,0.10.b,1.3.2,calc_elastic_constants_static,main,finished,3 Mar 2020,dff08f9e-e9c4-4b33-b64a-4a862d5dcf8c,...,"[0, 0, 1]",0.0,0.000001,5000,10000,0.01,1.000000e-08,[[1.17290e+00 9.84607e-01 9.84607e-01 0.00000e...,"[[1.1728960803189092, 0.9846066610359576, 0.98...","[[1.1728960491101497, 0.9846067371903183, 0.98..."
3,7db15eac-e174-4152-8d27-d4424a2959ea,7db15eac-e174-4152-8d27-d4424a2959ea,https://potentials.nist.gov/pid/rest/local/pot...,0.10.b,1.3.2,calc_elastic_constants_static,main,finished,3 Mar 2020,dff08f9e-e9c4-4b33-b64a-4a862d5dcf8c,...,"[0, 0, 1]",0.0,0.000001,5000,10000,0.01,1.000000e-07,[[1.1729 0.98461 0.98461 0. 0. 0. ...,"[[1.1728964394570842, 0.9846065927577837, 0.98...","[[1.1728956776809167, 0.9846067932985939, 0.98..."
4,8070b5e7-3c40-48cb-91b3-2738c11b4f05,8070b5e7-3c40-48cb-91b3-2738c11b4f05,https://potentials.nist.gov/pid/rest/local/pot...,0.10.b,1.3.2,calc_elastic_constants_static,main,finished,3 Mar 2020,dff08f9e-e9c4-4b33-b64a-4a862d5dcf8c,...,"[0, 0, 1]",0.0,0.000001,5000,10000,0.01,1.000000e-06,[[1.17288 0.98474 0.98474 0. 0. 0. ...,"[[1.172881008324584, 0.9847342137308721, 0.984...","[[1.1728733159142901, 0.9847361380131118, 0.98..."
5,8a3b31df-a6bb-4b95-bb3c-c2694412ef9b,8a3b31df-a6bb-4b95-bb3c-c2694412ef9b,https://potentials.nist.gov/pid/rest/local/pot...,0.10.b,1.3.2,calc_elastic_constants_static,main,finished,3 Mar 2020,dff08f9e-e9c4-4b33-b64a-4a862d5dcf8c,...,"[0, 0, 1]",0.0,0.000001,5000,10000,0.01,1.000000e-06,[[1.1729 0.98461 0.98461 0. 0. 0. ...,"[[1.1728999017095958, 0.9846057290521923, 0.98...","[[1.1728922153036136, 0.9846076572790847, 0.98..."


In [48]:
for record in records:
    print(uc.get_in_units(record.C.Cij[0,0], 'GPa'))

187.9156382400132
187.91563985007238
187.9186668996171
187.91866591499482
187.91563836999921
187.91866590499788
